In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Read in dataset

In [2]:
rrs_xds = xr.open_dataset('/glusteruser/awindled/all_NOBM_OASIM_data_2020.nc', decode_times=False)
rrs_xds

<xarray.Dataset>
Dimensions:         (months: 12, lat: 234, lon: 288, wavelength: 401)
Coordinates:
  * months          (months) float32 0.0 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * lat             (lat) float32 -84.0 -83.33 -82.67 -82.0 ... 70.0 70.67 71.33
  * lon             (lon) float32 -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * wavelength      (wavelength) int64 350 351 352 353 354 ... 747 748 749 750
    wv              (wavelength) int64 ...
Data variables: (12/42)
    tot             (months, lat, lon) float32 ...
    chl             (months, lat, lon) float32 ...
    dia             (months, lat, lon) float32 ...
    coc             (months, lat, lon) float32 ...
    cya             (months, lat, lon) float32 ...
    din             (months, lat, lon) float32 ...
    ...              ...
    coc_a           (wavelength, months, lat, lon) float32 ...
    coc_bb          (wavelength, months, lat, lon) float32 ...
    cdoc_a          (wavelength, months, lat, lon) float32 ...
    det_a           (wavelength, months, lat, lon) float32 ...
    det_bb          (wavelength, months, lat, lon) float32 ...
    pic_bb          (wavelength, months, lat, lon) float32 ...

# Calculate and add IOP ratios

In [3]:
#ratio as an index of flattening of the spectra 
rrs_xds['aph440_aph676'] = rrs_xds.total_phy_a[90,:,:,:] / rrs_xds.total_phy_a[326,:,:,:]

#estimate of the contribution of phytoplankton to the total pool of particles
rrs_xds['aph440_ap440'] = rrs_xds.total_phy_a[90,:,:,:] / rrs_xds.particulate_a[90,:,:,:]

#cdom absorption at 443
rrs_xds['a_cdoc_443'] = rrs_xds.cdoc_a[93,:,:,:]

#bbp slope using 440 and 555
rrs_xds['bbp_s_440_555'] = rrs_xds.particulate_bb[90,:,:,:] / rrs_xds.particulate_bb[205,:,:,:]

#bbp/bp - sensitive to particulate composition 
#rrs_xds['bbp_bp_443'] = total_phy_bb.440 / 530
#Q: Should I do this one since GIOP doesn't produce scattering? 

rrs_xds

<xarray.Dataset>
Dimensions:         (months: 12, lat: 234, lon: 288, wavelength: 401)
Coordinates:
  * months          (months) float32 0.0 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * lat             (lat) float32 -84.0 -83.33 -82.67 -82.0 ... 70.0 70.67 71.33
  * lon             (lon) float32 -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * wavelength      (wavelength) int64 350 351 352 353 354 ... 747 748 749 750
    wv              (wavelength) int64 ...
Data variables: (12/46)
    tot             (months, lat, lon) float32 ...
    chl             (months, lat, lon) float32 ...
    dia             (months, lat, lon) float32 ...
    coc             (months, lat, lon) float32 ...
    cya             (months, lat, lon) float32 ...
    din             (months, lat, lon) float32 ...
    ...              ...
    det_bb          (wavelength, months, lat, lon) float32 ...
    pic_bb          (wavelength, months, lat, lon) float32 ...
    aph440_aph676   (months, lat, lon) float32 nan nan nan nan ... nan nan nan
    aph440_ap440    (months, lat, lon) float32 nan nan nan nan ... nan nan nan
    a_cdoc_443      (months, lat, lon) float32 ...
    bbp_s_440_555   (months, lat, lon) float32 nan nan nan nan ... nan nan nan

# Normalize rrs and add as new variable

In [4]:
rrs_xds['rrs_norm'] = (rrs_xds.rrs - rrs_xds.rrs.mean()) / rrs_xds.rrs.std()
rrs_xds

<xarray.Dataset>
Dimensions:         (months: 12, lat: 234, lon: 288, wavelength: 401)
Coordinates:
  * months          (months) float32 0.0 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * lat             (lat) float32 -84.0 -83.33 -82.67 -82.0 ... 70.0 70.67 71.33
  * lon             (lon) float32 -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * wavelength      (wavelength) int64 350 351 352 353 354 ... 747 748 749 750
    wv              (wavelength) int64 ...
Data variables: (12/47)
    tot             (months, lat, lon) float32 ...
    chl             (months, lat, lon) float32 ...
    dia             (months, lat, lon) float32 ...
    coc             (months, lat, lon) float32 ...
    cya             (months, lat, lon) float32 ...
    din             (months, lat, lon) float32 ...
    ...              ...
    pic_bb          (wavelength, months, lat, lon) float32 ...
    aph440_aph676   (months, lat, lon) float32 nan nan nan nan ... nan nan nan
    aph440_ap440    (months, lat, lon) float32 nan nan nan nan ... nan nan nan
    a_cdoc_443      (months, lat, lon) float32 ...
    bbp_s_440_555   (months, lat, lon) float32 nan nan nan nan ... nan nan nan
    rrs_norm        (wavelength, months, lat, lon) float64 nan nan ... nan nan

# Prediction of U (scores) of samples using the V (loadings) of training PCA

In [5]:
# Load PCA loadings (calculated elsewhere) 
loadings_trun = pd.read_csv('/glusteruser/awindled/loadings_trun.csv', index_col=0) 
loadings_trun

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8
0,0.055005,0.055595,0.056184,0.056776,0.057372,0.057974,0.058585,0.059204
1,-0.049796,-0.049860,-0.049911,-0.049952,-0.049987,-0.050018,-0.050051,-0.050215
2,0.098926,0.098782,0.098637,0.098496,0.098354,0.098211,0.098068,0.097807
3,-0.120409,-0.119454,-0.118471,-0.117459,-0.116416,-0.115342,-0.114230,-0.113238
4,0.123148,0.121144,0.119051,0.116859,0.114560,0.112167,0.109646,0.105997
...,...,...,...,...,...,...,...,...
396,0.006164,-0.000132,0.003332,-0.001595,0.000212,-0.005240,-0.000307,-0.002313
397,-0.001991,0.002591,0.001718,-0.001795,0.000579,-0.003002,0.000490,0.000445
398,0.003266,0.004642,0.002250,-0.003100,-0.004262,-0.000818,-0.003591,-0.000376
399,-0.002043,-0.000110,-0.001774,0.004298,-0.000565,0.003607,0.002535,-0.000262


In [6]:
#Have to stack dataset for calculation
rrs_xds_norm_stacked = rrs_xds.rrs_norm.stack(z=("months", "lat", "lon")).T
rrs_xds_norm_stacked

<xarray.DataArray 'rrs_norm' (z: 808704, wavelength: 401)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * wavelength  (wavelength) int64 350 351 352 353 354 ... 746 747 748 749 750
    wv          (wavelength) int64 ...
  * z           (z) object MultiIndex
  * months      (z) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 11.0 11.0 11.0 11.0 11.0
  * lat         (z) float32 -84.0 -84.0 -84.0 -84.0 ... 71.33 71.33 71.33 71.33
  * lon         (z) float32 -180.0 -178.8 -177.5 -176.2 ... 176.2 177.5 178.8

In [9]:
%%time
## Prediction of U (scores) of samples using the V (loadings) 
#of the training PCA

r_sum = []
for j in range(len(loadings_trun)):
    for i in range(len(rrs_xds_norm_stacked)):
        r = loadings_trun.iloc[:,j] * rrs_xds_norm_stacked[i,:].values
        r_sum.append(np.sum(r))

IndexError: single positional indexer is out-of-bounds

In [10]:
len(r_sum)

6469632

In [11]:
6469632/808704

8.0

In [24]:
#create table of scores for each Rrs spectra (pixel)
RrsU_mod = [r_sum[x:x+808704] for x in range(0, len(r_sum),808704)]
RrsU_mod = pd.DataFrame(np.array(RrsU_mod)).T
RrsU_mod = RrsU_mod.set_axis(loadings_trun.columns, axis=1)
RrsU_mod 

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
808699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
808700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
808701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
808702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
RrsU_mod['PC1'].mean()

0.8457339234084528

In [36]:
RrsU_mod['PC4'].mean()

-0.21728813615950066

# save RrsU_mod as .csv 

In [28]:
#RrsU_mod.to_csv('/glusteruser/awindled/RrsU_mod.csv')

RrsU_mod = pd.read_csv('/glusteruser/awindled/RrsU_mod.csv').iloc[:,1:]
RrsU_mod

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
808699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
808700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
808701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
808702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Reshape RrsU_mod and add each PC score as variable to dataset

In [30]:
RrsU_mod_reshape = RrsU_mod.values.reshape(12, 234, 288, 8)
RrsU_mod_reshape.shape

(12, 234, 288, 8)

In [40]:
for i in range(len(RrsU_mod.columns)):
    rrs_xds['PC_'+str(i+1)] = xr.DataArray(RrsU_mod_reshape[:,:,:,i], dims=('months', 'lat', 'lon'))
rrs_xds    

<xarray.Dataset>
Dimensions:         (months: 12, lat: 234, lon: 288, wavelength: 401)
Coordinates:
  * months          (months) float32 0.0 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * lat             (lat) float32 -84.0 -83.33 -82.67 -82.0 ... 70.0 70.67 71.33
  * lon             (lon) float32 -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * wavelength      (wavelength) int64 350 351 352 353 354 ... 747 748 749 750
    wv              (wavelength) int64 ...
Data variables: (12/55)
    tot             (months, lat, lon) float32 ...
    chl             (months, lat, lon) float32 ...
    dia             (months, lat, lon) float32 ...
    coc             (months, lat, lon) float32 ...
    cya             (months, lat, lon) float32 ...
    din             (months, lat, lon) float32 ...
    ...              ...
    PC_3            (months, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PC_4            (months, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PC_5            (months, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PC_6            (months, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PC_7            (months, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    PC_8            (months, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [41]:
rrs_xds['PC_1'].mean()

<xarray.DataArray 'PC_1' ()>
array(0.84573392)

In [42]:
rrs_xds['PC_4'].mean()

<xarray.DataArray 'PC_4' ()>
array(-0.21728814)

# Save new dataset w/ score variables

In [43]:
xr.Dataset(rrs_xds).to_netcdf('/glusteruser/awindled/ALL_NOBM_OASIM_data_2020_w_scores.nc', mode='w', format='NETCDF4')
